<a href="https://colab.research.google.com/github/alxfed/colab/blob/master/Discover_socrata_datasets_and_get_metadata_sf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Discover Socrata SF datasets and get their metadata

In [0]:
from requests import Request, Session
import pandas

DISCOVERY_API_URL = 'http://api.us.socrata.com/api/catalog/v1'
CHICAGO_RESOURCE_URI = 'data.sfgov.org'

def all_sf_datasets():
    session = Session()
    # both, the domain and the context should be the same URI here
    params = {'domains': CHICAGO_RESOURCE_URI, 'search_context': CHICAGO_RESOURCE_URI}
    request = Request(method='GET', url=DISCOVERY_API_URL, params=params)
    prepped = request.prepare()
    response = session.send(prepped)
    return response.json()

In [4]:
all_datasets = all_sf_datasets()['results']
print(len(all_datasets))

100


In [0]:
from datetime import datetime

sections = ['resource', 'classification', 'metadata', 'permalink', 'link', 'preview_image_url', 'owner']

catalog = pandas.DataFrame()
for ind, dataset in enumerate(all_datasets):
  row = dict()
  complete_row = dataset['resource']
  row['id']                 = complete_row['id']
  row['name']               = complete_row['name']
  row['updatedAt'] = datetime.strptime(complete_row['updatedAt'], '%Y-%m-%dT%H:%M:%S.000Z')
  row['createdAt'] = datetime.strptime(complete_row['createdAt'], '%Y-%m-%dT%H:%M:%S.000Z')
  row['description'] = complete_row['description'].replace('<div>', '') # some f...ing 'web' f...ing 'master' showed a tag into a text field
  row['columns_name'] = ','.join(complete_row['columns_name'])
  row['columns_field_name'] = ','.join(complete_row['columns_field_name'])
  row['columns_datatype']   = ','.join(complete_row['columns_datatype'])
  row['columns_description']= ','.join(complete_row['columns_description'])
  catalog = catalog.append(row, ignore_index=True)

In [0]:
columns_order = ['id', 'name', 'createdAt', 'updatedAt', 'description', 'columns_name', 'columns_field_name', 'columns_datatype', 'columns_description']
catalog = catalog.reindex(columns=columns_order)

catalog.head(n=3)

,id,name,createdAt,updatedAt,description,columns_name,columns_field_name,columns_datatype,columns_description
0,5cd6-ry5g,Crimes - 2001 to present - Dashboard,2015-04-04 02:23:43,2020-02-03 12:11:28,This dataset reflects reported incidents of cr...,,,,
1,xzkq-xp2w,"Current Employee Names, Salaries, and Position...",2011-09-27 20:18:55,2020-01-31 23:21:48,This dataset is a listing of all current City ...,"Name,Job Titles,Department,Full or Part-Time,S...","name,job_titles,department,full_or_part_time,s...","text,text,text,text,text,number,number,number","Name of employee,Title of employee at the time..."
2,ydr8-5enu,Building Permits,2011-09-30 12:00:08,2020-02-03 13:04:32,INFORMATION ABOUT 7/12/2019 CHANGES TO THIS DA...,"ID,PERMIT#,PERMIT_TYPE,REVIEW_TYPE,APPLICATION...","id,permit_,permit_type,review_type,application...","text,text,text,text,calendar_date,calendar_dat...","Unique database record identifier,Tracking num..."


# Upload a *.sqlite file for the database and store the metadata there

In [0]:
import sqlalchemy as sqlalc

DATABASE_URI = 'sqlite:///colab.sqlite' # this time I named the file colab.sqlite
DATASETS_META = 'datasets_metadata' # this is the name of the table in the database

conn = sqlalc.create_engine(DATABASE_URI)

In [0]:
catalog.to_sql(name=DATASETS_META, con=conn, if_exists='replace', index=False)

# Download the *.sqlite file and put it into the git-lfs repo
You have to initiate git lfs in your repo before adding the file to it:
<pre>
git lfs install
git lfs track "*.sqlite"
git add .gitattributes
</pre>

In [0]:
from google.colab import files

with open('dummy.txt', 'w') as f:
  f.write('nothing')

files.download('dummy.txt') # without a dummy file (and request for permission from the browser)
files.download('colab.sqlite') # it doesn't download a binary